In [1]:
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *

In [33]:
rfiles = glob('%s/%s/*_0.txt' % (DPATH, ROUTES))
print(len(rfiles))

328


In [42]:
all_routes = []
for rf in rfiles:
    with open(rf) as fl:
        stops = fl.read().split('\n')[:-1]
    if len(stops) > 5:
        all_routes.append(dict(name=fileName(rf), stops=stops))

In [43]:
slens = [len(st['stops']) for st in all_routes]
print(max(slens), np.mean(slens), min(slens))
print(len(all_routes))

114 39.61180124223603 7
322


In [44]:
segfiles = glob('%s/%s/**/*.csv' % (DPATH, SPEEDS))
print(len(segfiles))
scheck = {}
for sf in segfiles:
    scheck[fileName(sf)] = True

16728


In [45]:
all_segs = []
for route in all_routes:
    stops = route['stops']
    sl = []
    for si in range(len(stops) - 1):
        sl.append(segkey(stops[si], stops[si+1]))
    route['segs'] = sl
    all_segs.append(route)

In [46]:
safe_segs = []
for route in all_segs:
    segs = route['segs']
    if all([entry in scheck for entry in segs]):
        safe_segs.append(route)
print(len(safe_segs))

245


In [50]:
print(safe_segs[0]['name'], safe_segs[0]['segs'])

Bx33_0 ['403717-404111', '404111-404113', '404113-403720', '403720-403721', '403721-403722', '403722-102185', '102185-103991', '103991-102187', '102187-102188', '102188-102189', '102189-102190', '102190-102191', '102191-102192', '102192-102193', '102193-102988', '102988-102195', '102195-104248', '104248-103284', '103284-101201', '101201-802102']


In [53]:
for route in safe_segs:
    with open('data/routes/%s.txt' % route['name'], 'w') as fl:
        fl.write('name,seg\n')
        for seg in route['segs']:
            fl.write('%s,%s\n' % (route['name'], seg))

In [64]:
for route in tqdm(safe_segs):
    vmat = -np.ones((13248, len(route['segs'])))
    for si, seg in enumerate(route['segs']):
        with open('%s/%s/%s/%s.csv' % (DPATH, SPEEDS, seg[0], seg)) as fl:
            _ = fl.readline() # header
            line = fl.readline()
            li = 0
            while line:
                line = line.strip('\n').split(',')
                vel = None if line[1] == '' else float(line[1])
                vmat[li, si] = vel
                line = fl.readline()
                li += 1
    savef = 'data/history/%s.npy' % route['name']
    np.save(savef, vmat)

100%|██████████| 245/245 [16:24<00:00,  4.02s/it]
